# 删除多余文件

## 1. 导入工具包

In [ ]:
# 导入操作系统库以获取平台信息以及对目录操作的权限
import os

# 导入OpenCV2库以支持图像处理操作
import cv2

# 导入图像处理库的图像模块以支持图像处理操作
from PIL import Image

# 导入计算扩展库以支持数组运算
import numpy

# 导入进度条库的模块以令进程可视化
from tqdm import tqdm

## 2. 删除系统自动生成的多余文件

- ### Linux系统

In [ ]:
'''
查找格式：find /（查找范围） -name '查找关键字' -type d
find后的 . 代表当前目录
使用 -inname 能够忽略字符大小
'''

# Find命令递归查找当前目录是否有用于Mac系统用户预览的缓存文件夹（文件类似于Windows系统的Thumbs.db）
!find . -iname '__MACOSX'
#删除
!for i in `find . -iname '__MACOSX'`; do rm -rf $i; done
# 检查是否已删除
!find . -iname '__MACOSX'


# Find命令递归查找当前目录是否有用于Mac系统桌面服务存储的文件
!find . -iname '.DS_Store'
#删除
!for i in `find . -iname '.DS_Store'`; do rm -rf $i; done
# 检查是否已删除
!find . -iname '.DS_Store'


# Find命令递归查找当前目录是否有用于jupyter进行定点恢复的文件
!find . -iname '.ipynb_checkpoints'
#删除
!for i in `find . -iname '.ipynb_checkpoints'`; do rm -rf $i; done
# 检查是否已删除
!find . -iname '.ipynb_checkpoints'

- ### Windows系统

In [ ]:
'''
查找格式：Get-ChildItem -Path '查找范围' "查找关键字"
-Path后的 . 代表当前目录
使用-Recurse参数以获取指定位置中的项目以及该位置的所有子项目。
使用-Filter参数以指定用于限定-Path参数的过滤器。
管道符 | 会把前一个命令的对象传递给第二个命令所对应支持ByValue的参数，然后执行
'''

# Get-ChildItem命令递归查找当前目录是否有用于Mac系统用户预览的缓存文件夹（文件类似于Windows系统的Thumbs.db）并递归强制删除
os.system('Get-ChildItem -Path . -Recurse -Filter "__MACOSX" | Remove-Item -Force')
# 检查是否已删除
os.system('Test-Path -Path . -Filter "__MACOSX"')


# Get-ChildItem命令递归查找当前目录是否有用于Mac系统桌面服务存储的文件
os.system('Get-ChildItem -Path . -Recurse -Filter ".DS_Store" | Remove-Item -Force')
# 检查是否已删除
os.system('Test-Path -Path . -Filter ".DS_Store"')


# Get-ChildItem命令递归查找当前目录是否有用于jupyter进行定点恢复的文件
os.system('Get-ChildItem -Path . -Recurse -Filter ".ipynb_checkpoints" | Remove-Item -Force')
# 检查是否已删除
os.system('Test-Path -Path . -Filter ".ipynb_checkpoints"')

- ### 其它

In [ ]:
# Python命令查找指定目录下是否有带目标后缀的文件
'%目标后缀%' in os.listdir('%指定目录%')

## 3. 删除gif格式的文件与非三通道的文件

- ### Unix / Windows 系统

In [ ]:
'''
img = cv2.imread(path) ：这是opencv中的处理图片的函数，使用时需 import cv2
img = Image.open(path) ：这是PIL中的一个处理图片的函数，使用时需 from PIL import Image （或者 Import PIL 之后 img = PIL.Image.open(path)）
同时需注意：
当图像格式为RGBA时，Image.open()读取的格式为RGBA（其中A表示图像的alpha通道，即RGBA共四个通道）；而cv2.imread()读取的格式是BGR，只有三个通道。故使用PIL进行处理非三通道检测处理。
'''

# 指定工作目录
work_dir = 'DataSet'
#work_dir = os.getcwd() # 指定根目录为工作目录

for subfolder in tqdm(os.listdir(work_dir)): # 遍历工作目录下包含的子文件夹并让进度可视化
    for file in os.listdir(os.path.join(work_dir, subfolder)): # 合并工作目录、子文件夹得到子文件夹路径以遍历路径下包含的文件
        file_path = os.path.join(work_dir, subfolder, file) # 合并工作目录、子文件夹、文件名得到文件路径

        # 先判断是否为gif文件
        image = cv2.imread(file_path, cv2.IMREAD_COLOR) # 以默认参数（彩色图片，忽略alpha通道）读入路径对应的图片
        if image is None: # 非gif图像文件通常返回为np.array类型，其余返回None
            print(file_path, '判定为gif文件，执行删除')
            os.remove(file_path)
        else:
            # 再对非gif文件判断是否为非三通道文件
            image = numpy.array(Image.open(file_path)) # 由于要使用img.shape函数，故打开图片后需将image形式转换成array数组
            try: # 尝试获取图像通道信息并判定通道数是否为三
                channel = image.shape[2] # [0]返回水平像素数；[1]返回垂直像素数；[2]返回通道数
                if channel != 3:
                    print(file_path, '判定为非三通道文件，执行删除')
                    os.remove(file_path)
            except:
                print(file_path, '不明文件，执行删除')
                os.remove(file_path)

## 4. 再次删除多余的`.ipynb_checkpoints`目录

- ### Linux系统

In [ ]:
# Find命令递归查找当前目录是否有用于jupyter进行定点恢复的文件
!find . -iname '.ipynb_checkpoints'
#删除
!for i in `find . -iname '.ipynb_checkpoints'`; do rm -rf $i; done
# 验证是否已删除
!find . -iname '.ipynb_checkpoints'

- ### Windows系统

In [ ]:
# Get-ChildItem命令递归查找当前目录是否有用于jupyter进行定点恢复的文件
os.system('Get-ChildItem -Path . -Recurse -Filter ".ipynb_checkpoints" | Remove-Item -Force')
# 检查是否已删除
os.system('Test-Path -Path . -Filter ".ipynb_checkpoints"')